In [1]:
import pandas as pd
import numpy as np
import sys
import os

import time

print(os.getcwd())
os.chdir('../../')
print(os.getcwd())






/home/tasnina/Projects/SynVerse/code/feature_extraction
/home/tasnina/Projects/SynVerse


In [2]:
dataset_dir = "datasets/"
drug_target_links_file = dataset_dir+ 'drug-comb-db/' + 'drug_protein_links.tsv'
target_file = dataset_dir+ 'drug-comb-db/' + 'targets.tsv'
STRINGID_uniprot_map_file = dataset_dir + 'mappings/human/'+ 'STRINGID_uniprot_map.tsv'
drug_target_map_ttd_drug_bank_file = 'inputs/drugs/drug_target_map_only_ttd_drug_bank.tsv' 
final_drug_target_map_file = 'inputs/drugs/drug_target_map.tsv' 

In [3]:
drug_target_df = pd.read_csv(drug_target_links_file, sep='\t')
print(drug_target_df.nunique())
#this is a one time thing. Save the STRINGID of target proteins in a file. 
#Not in code: Later upload the file to https://www.uniprot.org/uploadlists/ to retrieve uniprotID.
#save the uniprotID's in STRINGID_uniprot_map.tsv
targets = pd.Series(drug_target_df['protein'].unique())
targets.to_csv(target_file, sep='\t', index = False, header = False)
print(len(targets))
print(len(drug_target_df))

chemical          787039
protein            19195
experimental         910
prediction           389
database              15
textmining           901
combined_score       850
dtype: int64
19195
15473939


In [4]:
pd.Series(drug_target_df['chemical'].unique()).to_csv(dataset_dir+ 'drug-comb-db/drugcombdb_drugs.csv', sep='\t')

In [ ]:
#read the STRING_ID to uniprot_id mapping file
string_to_uniprot_df = pd.read_csv(STRINGID_uniprot_map_file, sep='\t')
print(string_to_uniprot_df.head())
string_to_uniprot_dict = dict(zip(string_to_uniprot_df['STRING'], string_to_uniprot_df['uniprot']))

#now read the drug_comb_db drug_target info file. Rename column 'chemical' as 'pubchem_id'
drug_target_df = pd.read_csv(drug_target_links_file, sep='\t')
drug_target_df.head()
drug_target_df['pubchem_cid'] = drug_target_df['chemical'].apply(lambda x: x.replace('CIDm',''))

#filter out the target proteins from drug_comb_db that do not have uniprot id mapping
drug_target_df= drug_target_df[drug_target_df['protein'].isin(string_to_uniprot_dict.keys())]
drug_target_df['uniprot_id'] = drug_target_df['protein'].apply(lambda x:string_to_uniprot_dict[x])                           
print(drug_target_df.head())

#only keep columns 'pubchem_cid' and 'uniprot_id'
drug_target_df = drug_target_df[['pubchem_cid','uniprot_id']]

#load drug target map containing targets from TTD and drugbank and combine it with target info from drugcombdb
init_drug_target_map_df = pd.read_csv(drug_target_map_ttd_drug_bank_file, sep='\t')
final_drug_target_df = pd.concat([init_drug_target_map_df,drug_target_df], axis=0).reset_index(drop=True)
final_drug_target_df.drop_duplicates(inplace=True)



In [ ]:
final_drug_target_df = final_drug_target_df[['pubchem_cid','uniprot_id']]
final_drug_target_df.to_csv(final_drug_target_map_file, sep='\t', index=False)
print(final_drug_target_df.head())

In [ ]:
#create the feature matrix containing all the targets for a drug as a 0-1 vector.
unique_proteins = final_drug_target_df['uniprot_id'].unique()
unique_proteins_in_Index_form = pd.Index(pd.Series(unique_proteins))
number_of_unique_proteins = len(unique_proteins)
unique_pubchem_ids = final_drug_target_df['pubchem_cid'].unique()

drug_targets ={ pubchem_id: np.zeros(number_of_unique_proteins, dtype=int) for pubchem_id in  unique_pubchem_ids}

for index,row in final_drug_target_df.iterrows():
    #find the index of the current target (i.e.present in the current row) in unique_proteins
    prot_position = unique_proteins_in_Index_form.get_loc(row['uniprot_id'])
    drug_targets[row['pubchem_cid']][prot_position] = 1
    
drug_target_df_feature_ready = pd.DataFrame.from_dict(drug_targets, orient='index', columns=unique_proteins)
drug_target_df_feature_ready = drug_target_df_feature_ready.reset_index().rename(columns={'index':'pubchem_cid'})
drug_target_df_feature_ready['pubchem_cid']=drug_target_df_feature_ready['pubchem_cid'].astype(str)
print(drug_target_df_feature_ready.head())
print(drug_target_df_feature_ready.shape)
print(drug_target_df_feature_ready['pubchem_cid'].unique())